In [3]:
import numpy as np
import matplotlib.pylab as plt
import math as math
from scipy.optimize import fmin
from iminuit import Minuit, describe, Struct
from scipy.spatial import distance
from scipy.sparse import vstack

import time

#import lichen as lch
import seaborn as sns

%matplotlib  notebook

In [4]:
# Add code to the following function so that it takes in two datasets, loops over one of them, and finds
# information about the nearest neighbors in the other dataset, based on a flag. 

def nn(data0,data1,r=None,nneighbors=None):
    
    ret = -1
    ret_list=[]
    if r is not None and nneighbors is not None:
        exit(-1)
        return ret
    
    
    
    elif r is not None and nneighbors is None:
        rsq = r*r
        #for num0,num1 in zip(data0[0],data0[1]):
        for d in data0.transpose():
            count=0
            diffx=d[0]-data1[0]
            diffy=d[1]-data1[1]
            #diff= np.sqrt((num0-data1[0])**2 +(num1-data1[1])**2)
            #diff= np.sqrt(diffx*diffx + diffy*diffy)
            diff= diffx*diffx + diffy*diffy

            count = len(diff[diff<rsq])
            ret_list.append(float(count)/(float(len(data1[0]))*r))
        ret_list = np.array(ret_list)
        #print ret_list
        #exit()
        return ret_list
    
    
    
    
    elif r is None and nneighbors is not None:
        for num0 in data0:
            diff = np.abs(num0 - data1)
            diff.sort()
            radius= diff[nneighbors-1]
            ret_list.append(1/radius)
        ret_list = np.array(ret_list)
        return ret_list
    return ret

In [5]:
# Add code to the following function so that it takes in two datasets, loops over one of them, and finds
# information about the nearest neighbors in the other dataset, based on a flag. 

def nncdist(data0,data1,r=None,nneighbors=None):
    
    ret = -1
    ret_list=[]
    if r is not None and nneighbors is not None:
        exit(-1)
        return ret
    
    
    
    elif r is not None and nneighbors is None:
        #combined= np.vstack((data0[0],data0[1])).T  
        #combined1= np.vstack((data1[0],data1[1])).T
        combined = data0.transpose()
        combined1 = data1.transpose()
        #print data0.transpose()
        dist=distance.cdist(combined,combined1,'euclidean')
        for num in dist:
            count=len(num[num<r])
            #count =len(dist[dist<r])
            ret_list.append(float(count)/(float(len(data1[0]))*r))
        ret_list = np.array(ret_list)
        return ret_list
    
    
    
    
    elif r is None and nneighbors is not None:
        for num0 in data0:
            diff = np.abs(num0 - data1)
            diff.sort()
            radius= diff[nneighbors-1]
            ret_list.append(1/radius)
        ret_list = np.array(ret_list)
        return ret_list
    return ret

In [6]:
nbkg = 1000
nMC=1000
sigmeans = [5.0,7.0]
sigwidths = [0.2,0.2]
bkglos = [3.5,5]
bkghis = [6,9]
frac=.1667




def signal_2D(npts,means,sigmas):
    pts = []
    for m,s in zip(means,sigmas):
        pts.append(np.random.normal(m,s,npts))
        
    pts = np.array(pts)
    return pts

# Flat in 2D
def background_2D(npts,lovals,hivals):
    pts = []
    for lo,hi in zip(lovals,hivals):
        width = hi-lo
        pts.append(lo + width*np.random.random(npts))
        
    pts = np.array(pts)
    return pts

def gen_sig_and_bkg(npts,means,sigmas,lovals,hivals):
    sigpts = signal_2D(npts[0],means,sigmas)
    bkgpts = background_2D(npts[1],lovals,hivals)

    data = [sigpts[0].copy(),sigpts[1].copy()]
    data[0] = np.append(data[0],bkgpts[0])
    data[1] = np.append(data[1],bkgpts[1])
    data = np.array(data)
   
    return data



In [7]:
#data1=np.array([[1, 1,5,6,7,100],[1, 1,44,5,1,7]])
#data2=np.array([[3, 3,7,1,3,88] ,[3, 3,71,7,8,6]])

data3 = gen_sig_and_bkg([200,1000],sigmeans,sigwidths,bkglos,bkghis)
signal_points= signal_2D(nMC,sigmeans,sigwidths)
background_points = background_2D(nMC,bkglos,bkghis)





r=.1

In [8]:
signal=nn(data3,signal_points,r)
signal_cdist=nncdist(data3,signal_points,r)

signal=nn(data3,background_points,r)
signal_cdist=nncdist(data3,background_points,r)

In [9]:
print signal
print signal_cdist

[ 0.01  0.01  0.02 ...,  0.02  0.03  0.01]
[ 0.01  0.01  0.02 ...,  0.02  0.03  0.01]
